In [2]:
import img2pdf
import glob
from fpdf import FPDF
from PIL import Image
import os
import ntpath
import ocrmypdf 
import numpy as np
import re
import pdfplumber
import pandas as pd
import time
import cv2
from matplotlib import pyplot as plt
from pdf2image import convert_from_path
pd.set_option('display.max_rows', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

C:\Users\mkahs\anaconda3\envs\IBR_DEMO\lib\site-packages\ipykernel_launcher.py:20: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.


In [10]:

#https://stackoverflow.com/questions/28816046/
#displaying-different-images-with-actual-size-in-matplotlib-subplot
def display(im_path):
    dpi = 80
    im_data = plt.imread(im_path)

    height, width  = im_data.shape[:2]
    
    # What size does the figure need to be in inches to fit the image?
    figsize = width / float(dpi), height / float(dpi)

    # Create a figure of the right size with one axes that takes up the full figure
    fig = plt.figure(figsize=figsize)
    ax = fig.add_axes([0, 0, 1, 1])

    # Hide spines, ticks, etc.
    ax.axis('off')

    # Display the image.
    ax.imshow(im_data, cmap='gray')

    plt.show()
    

    
################ GRAY SCALE CONVERSION ###########    
def grayscale(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)    



################ Noise Removal ###########
def noise_removal(image):
    import numpy as np
    kernel = np.ones((1, 1), np.uint8)
    image = cv2.dilate(image, kernel, iterations=1)
    kernel = np.ones((1, 1), np.uint8)
    image = cv2.erode(image, kernel, iterations=1)
    image = cv2.morphologyEx(image, cv2.MORPH_CLOSE, kernel)
#     image = cv2.medianBlur(image, 3)
    return (image)

def morph_open(image):
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1,2))
    morphology_img = cv2.morphologyEx(image, cv2.MORPH_OPEN, kernel,iterations=1)
    return (morphology_img)
#     plt.imshow(morphology_img,'Greys_r')

########### Dilation and Erosion ##########
#######
#######
#######
########### Making the front THIN ########
def thin_font(image):
    import numpy as np
    image = cv2.bitwise_not(image)
    kernel = np.ones((2,2),np.uint8)
    image = cv2.erode(image, kernel, iterations=1)
    image = cv2.bitwise_not(image)
    return (image)



########### Making the front THICK ########
def thick_font(image):
    import numpy as np
    image = cv2.bitwise_not(image)
    kernel = np.ones((2,2),np.uint8)
    image = cv2.dilate(image, kernel, iterations=1)
    image = cv2.bitwise_not(image)
    return (image)




########### DETECTING SKEW ########
def getSkewAngle(cvImage) -> float:
    # Prep image, copy, convert to gray scale, blur, and threshold
    newImage = cvImage.copy()
    gray = cv2.cvtColor(newImage, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (9, 9), 0)
    thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

    # Apply dilate to merge text into meaningful lines/paragraphs.
    # Use larger kernel on X axis to merge characters into single line, cancelling out any spaces.
    # But use smaller kernel on Y axis to separate between different blocks of text
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (30, 5))
    dilate = cv2.dilate(thresh, kernel, iterations=2)

    # Find all contours
    contours, hierarchy = cv2.findContours(dilate, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key = cv2.contourArea, reverse = True)
    for c in contours:
        rect = cv2.boundingRect(c)
        x,y,w,h = rect
        cv2.rectangle(newImage,(x,y),(x+w,y+h),(0,255,0),2)

    # Find largest contour and surround in min area box
    largestContour = contours[0]
    print (len(contours))
    minAreaRect = cv2.minAreaRect(largestContour)
#     cv2.imwrite("temp/boxes.jpg", newImage)
    # Determine the angle. Convert it to the value that was originally used to obtain skewed image
    angle = minAreaRect[-1]
    if angle < -45:
        angle = 90 + angle
    return -1.0 * angle




############## Rotate the image around its center ###############
def rotateImage(cvImage, angle: float):
    newImage = cvImage.copy()
    (h, w) = newImage.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    newImage = cv2.warpAffine(newImage, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
    return newImage



############### Deskew image ###############
def deskew(cvImage):
    angle = getSkewAngle(cvImage)
    return rotateImage(cvImage, -1.0 * angle)



############### REMMOVING BORDERS ###############
def remove_borders(image):
    contours, heiarchy = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cntsSorted = sorted(contours, key=lambda x:cv2.contourArea(x))
    cnt = cntsSorted[-1]
    x, y, w, h = cv2.boundingRect(cnt)
    crop = image[y:y+h, x:x+w]
    return (crop)

################## CONVERT PDF TO LINES ##################

def Pdf_to_lines(pdf_file_path):
    raw_lines= {}
    total_line_cnt=0
    with pdfplumber.open(pdf_file_path) as pdf:
        page_count=pdf.pages[:]
#         print(len(page_count))
        for i in range (0,len(page_count)):
            page =pdf.pages[i]
            page_nm=i+1
    #         print(page_nm)
    #         print(page.page_number)
            text = page.extract_text()
            raw = text.split('\n')
            for i in range(0, len(raw)):
                line_nm=i+1
                total_line_cnt=total_line_cnt+1
                k= "Page-"+str(page_nm)+"_Line-"+str(line_nm)
#                 k= "Page-"+str(page_nm)+"_Line-"+str(line_nm)+"_TL-"+str(total_line_cnt)
                raw_lines[k]= raw[i]
    return raw_lines



############# Make PDF Searchable Lines #############


# timestr = time.strftime("%m%d%Y-%H%M%S")

def PDF_searchable_lines(input_file):
    timestr = time.strftime("%m%d%Y-%H%M%S")
    print(input_file)
    PDF_filename = input_file
#     PDF_filename = str(input_file)+str('.pdf')
    input_dir = "C:/Users/mkahs/Repository/IBR-main/IBR-main/NEW/image_processed_pdf/"
    input_file_dir= str(input_dir)+ str(PDF_filename)
    print("Input FileName: ", PDF_filename)
    print("Input Dir: ", input_file_dir)
    output_loc = "C:/Users/mkahs/Repository/IBR-main/IBR-main/NEW/searchable_pdf/"
#     timestr = time.strftime("%m%d%Y-%H%M%S")
    # print(PDF_filename)
    filename=PDF_filename.rsplit(".",1)[:-1]
    # print(str(filename[0]))
#     csv_path='/n01/data/nlp_aeac/mostofa_env/INPUT/CONVERTED_CSV_OUTPUT/'
    output_name= output_loc+filename[0]+'_'+str(timestr)+'.pdf'
    print("Output FIle: ", output_name)
    ocrmypdf.ocr(input_file_dir, output_name,rotate_pages=True, deskew=True, force_ocr=True)
    output_lines= Pdf_to_lines(output_name)
    return output_lines, timestr

########################## DATE EXTRACTION ################################


def date_detection(input_line_string):
    pref_num = re.compile(r'(\d{1,2}\/\d{1,2}\/\d{4}|\d{1,2}\/\d{1,2}\/\d{2}|\d{1,2}\-\d{1,2}\-\d{4}|\d{1,2}\-\d{1,2}\-\d{2}|\d{1,2}\d{1,2}\d{4}|\d{1,2}\d{1,2}\d{1,4}_(\d+)_)')
    m = pref_num.search(input_line_string)
    
    if m != None:
#         print(m.group(1))
#         print(m.string)
        return m.group(1)
    else:
        return "N/A"
        
# date_detection(l_s)




######################## DESCRIPTION EXTRACTION ##############################


# l_s='0250  |022621|  7955  |250  DEXTROSE  5  %  AND  0.9  %  SODIUM  1  15.00 '

def description_detection(input_line_string):
    loc=[]
    terms=[]
    pattern = r'([A-Za-z]+[\w@]*)'
    for match in re.finditer(pattern, input_line_string):
        s = match.start()
        e = match.end()
#         print('String match : ',(input_line_string[s:e], s, e))
        terms.append(input_line_string[s:e])
        loc.append(s)
        loc.append(e)
    split_loc=[loc[0], loc[-1]]
    truncated_text = input_line_string[loc[0]:loc[-1]]
#     print(truncated_text)
    return truncated_text

        





############################# DOLLAR EXTRACTION #####################


# description_detection(l_s)
# dollar_type_1 = '\d{1,2}[,\.]\d{1,3}[,\.]\d{2}'
# dollar_type_2 = '\d{1,1}[,\.]\d{1,3}[,\.]\d{2}'
# dollar_type_3 = '\d{1,3}[,\.]\d{1,3}[,\.]\d{2}'
# dollar_type_4 = '\d{1,2}[,\.]\d{1,3}[,\.]\d{1}'
# dollar_type_5 = '\d{1,1}[,\.]\d{1,3}[,\.]\d{1}'
# dollar_type_6 = '\d{1,3}[,\.]\d{1,3}[,\.]\d{1}'
# dollar_type_3 = '\d+[.]\d+'

# l_s='Page-1_Line-34 2500000  HEMOSTATIC  MATRIX $.30'

def dollar_detection(input_line_string):
    pref_num = re.findall('(\d{1,2}[,\.]\d{1,3}[,\.]\d{2}|d{1,1}[,\.]\d{1,3}[,\.]\d{2}|\d{1,3}[,\.]\d{1,3}[,\.]\d{2}|\d{1,2}[,\.]\d{1,3}[,\.]\d{1}|\d{1,1}[,\.]\d{1,3}[,\.]\d{2}|\d{1,3}[,\.]\d{1,3}[,\.]\d{1}|\d+[,\.]\d+|_(\d+)_)', input_line_string, re.DOTALL)
#     m = pref_num.search(input_line_string)
    match_length=len(pref_num)
    
   
    if match_length > 0:
        m=pref_num[-1]
#         print(m.group(1))
        s =m[0]
        if s[-2]==',':
            s = s.replace(',', '.')
        if s[-3]==',':
            s = s.replace(',', '.')
        s1 = s.replace(',', '')
#         print(s1)
        n=float(s1)
        return n
    else:
#         print('NA')
        return 0
        
# quant=dollar_detection(l_s)
# print(quant)

########################## DESCRIPTION EXTRACTION NEW_TEST ############################


# l_s='0250  |022621|  7955  |250  DEXTROSE  5  %  AND  0.9  %  SODIUM  1  15.00 '

def description_detection_1(input_line_string):
    loc=[]
    terms=[]
    pattern = r'([A-Za-z]+[\w@]*)'
    pref_num = re.compile(r'([A-Za-z]+[\w@]*)')
    m = pref_num.search(input_line_string)
    if m != None:
        for match in re.finditer(pattern, input_line_string):
            s = match.start()
            e = match.end()
    #         print('String match : ',(input_line_string[s:e], s, e))
            terms.append(input_line_string[s:e])
            loc.append(s)
            loc.append(e)
        split_loc=[loc[0], loc[-1]]
        truncated_text = input_line_string[loc[0]:loc[-1]]
    #     print(truncated_text)
        return truncated_text
    else:
        return 'No Description'

        
# description_detection(l_s)




################# DOLLAR EXTRACTION NEW_TEST ##########################




# dollar_type_1 = '\d{1,2}[,\.]\d{1,3}[,\.]\d{2}'
# dollar_type_2 = '\d{1,1}[,\.]\d{1,3}[,\.]\d{2}'
# dollar_type_3 = '\d{1,3}[,\.]\d{1,3}[,\.]\d{2}'
# dollar_type_4 = '\d{1,2}[,\.]\d{1,3}[,\.]\d{1}'
# dollar_type_5 = '\d{1,1}[,\.]\d{1,3}[,\.]\d{1}'
# dollar_type_6 = '\d{1,3}[,\.]\d{1,3}[,\.]\d{1}'
# dollar_type_3 = '\d+[.]\d+'

# l_s='Page-1_Line-34 2500000  HEMOSTATIC  MATRIX $.30'

def dollar_detection_1(input_line_string):
    pref_num = re.compile(r'(\d{1,2}[,\.]\d{1,3}[,\.]\d{2}|d{1,1}[,\.]\d{1,3}[,\.]\d{2}|\d{1,3}[,\.]\d{1,3}[,\.]\d{2}|\d{1,2}[,\.]\d{1,3}[,\.]\d{1}|\d{1,1}[,\.]\d{1,3}[,\.]\d{2}|\d{1,3}[,\.]\d{1,3}[,\.]\d{1}|\d+[.]\d+|_(\d+)_)')
    m = pref_num.search(input_line_string)
    
    if m != None:
#         print(m.group(1))
        s =m.group(1)
        if s[-2]==',':
            s = s.replace(',', '.')
        if s[-3]==',':
            s = s.replace(',', '.')
        s1 = s.replace(',', '')
#         print(s1)
        n=float(s1)
        return n
    else:
#         print('NA')
        return 'NaN'
        
# quant=dollar_detection(l_s)
# print(quant)



############ Search Term to DataFrame and Total Amount #########




def Search_in_PDF(term):
    search_term = str(term)
    t = []
    sl =[]
    date_l =[]
    des_l=[]
    search_l=[]
    position_l =[]
    flag_l=[]
    line_l=[]
    for key, val in clean_searchable_pdf_lines.items():
        if search_term in val:
            sd = key + " " + " : " + val
            line_l.append(sd)
            sl.append(re.split(r'\s+', sd))

            date_strip =date_detection(val)
            description = description_detection_1(val)
            dollar=dollar_detection(val)

            if dollar!=0:
                t.append(dollar)
                flag_l.append("")
            else:
                t.append(0)
                flag_l.append("Check the Amount")
            date_l.append(date_strip)
            des_l.append(description)
            search_l.append(search_term)
            position_l.append(key)

    print(t)
    total_amount =np.sum(t)
    print("Total Amount : ",total_amount) 
    # print(sl)
    clean_data= pd.DataFrame(
        {'Position': position_l,
        'Search_Term': search_l,
         'Date': date_l,
         'Description': des_l,
         'Amount': t,
         'FLAG' :flag_l,
         'Line_info' : line_l
        })
    return t, total_amount, clean_data




############## Save the Dataframe ###############




def save_SearchTerm_DF_to_CSV(filename, search_key, df, timestr):
    csv_path='C:/Users/mkahs/Repository/IBR-main/IBR-main/NEW/OUTPUT_CSV_TEMP/'
    raw_output_name= csv_path+"RAW_Search_"+str(search_key)+"_"+filename+'_'+str(timestr)+'.csv'
    print('Saved as: --',raw_output_name)
    df.to_csv(raw_output_name, sep=',', index=False)

    



############## Search Term to DataFrame and Total Amount ###############

def PDF_to_DF(start_df,end_df):

    t = []
    sl =[]
    date_l =[]
    des_l=[]
    line_text=[]
#     search_l=[]
    position_l =[]
    flag_l=[]
    
            
        
    for key, val in clean_searchable_pdf_lines.items():
        start_TL = key.split('_')[2]
        start_TL=start_TL.split('-')[1]
        start_TL=int(start_TL)
        if start_TL in range(start_df,end_df):
            print(start_TL)
            print(val)
            line_text.append(val)
            sd = key + " " + val
            sl.append(re.split(r'\s+', sd))

            date_strip =date_detection(val)
            description = description_detection_1(val)
            dollar=dollar_detection_1(val)

            if dollar!=0:
                t.append(dollar)
                flag_l.append("")
            else:
                t.append(0)
                flag_l.append("Check the Amount")
            date_l.append(date_strip)
            des_l.append(description)
            position_l.append(key)

#     print(t)
#     total_amount =np.sum(t)
#     print("Total Amount : ",total_amount) 
    # print(sl)
    clean_data= pd.DataFrame(
        {'Position': position_l,
         'Date': date_l,
         'Text': line_text,
         'Description': des_l,
         'Amount': t,
         'FLAG' :flag_l
        })
    return t, clean_data





def trigger_p(image):
    hist = cv2.calcHist([image],[0],None,[255],[0,255])
    within = []
    for i in range(len(hist)):
        x,y = np.split(hist,[i])
        x1 = np.sum(x)/(image.shape[0]*image.shape[1])
        y1 = np.sum(y)/(image.shape[0]*image.shape[1])
        x2 = np.sum([ j*t for j,t in enumerate(x)])/np.sum(x)
        y2 = np.sum([ j*t for j,t in enumerate(y)])/np.sum(y)
        x3 = np.sum([ (j-x2)**2*t for j,t in enumerate(x)])/np.sum(x)
        x3 = np.nan_to_num(x3)
        y3 = np.sum([ (j-y2)**2*t for j,t in enumerate(y)])/np.sum(y)
        y3 = np.nan_to_num(y3)
        within.append(x1*x3 + y1*y3)
    m = np.argmin(within)
    print(m)
    return m

#############################################################
##############################################################
########### PDF TO IMAGE PROCESSING #######################
#####
####

def pdf_to_image(s):
    upload_loc='C:/Users/mkahs/Repository/IBR-main/IBR-main/NEW/pdf_uploads/'
    outputLoc = 'C:/Users/mkahs/Repository/IBR-main/IBR-main/NEW/pdf_image_processed/'
    outputLoc_1 = 'C:/Users/mkahs/Repository/IBR-main/IBR-main/NEW/pdf_image_temp/'
    filename = s.replace(upload_loc,"")
    print(filename)
    img_name=filename[:-4]
    print(img_name)
    inputLoc = s
    ocrLoc = outputLoc+filename
    ocrLoc_1 = outputLoc_1+filename
    if not os.path.exists(ocrLoc_1):
        os.makedirs(ocrLoc_1)
        os.makedirs(ocrLoc)
    dpi=200
    pages = convert_from_path(inputLoc ,dpi )
    print("New File Transfer........")
    for i in range(len(pages)):
        page = pages[i]
        page.save('{}/{}_{}.jpg'.format(ocrLoc_1,img_name,i), 'JPEG')
        new_im=ocrLoc_1+"/"+img_name+"_"+str(i)+".jpg"
        print(new_im)
        img = cv2.imread(new_im)
#         img=deskew(img)
        gray_image = grayscale(img)
        t_p = trigger_p(gray_image)
        if (t_p < 128):
            gray_image = noise_removal(gray_image)
            print('NOISE REMOVAL DONE')
            
#         morph=morph_open(gray_image)
        kernel_dilation = np.ones((1,1), np.uint8)
#         gray_image=deskew(gray_image)
        ret, cv_otsu_binary = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        dilation = cv2.dilate(cv_otsu_binary, kernel_dilation, iterations=1)
        cv2.imwrite('{}/{}_{}.jpg'.format(ocrLoc,img_name,i), dilation)
    return ocrLoc, filename


#############################################################
##############################################################
########### IMAGE TO PDF PROCESSING #######################
#####
####


def image_to_pdf(input_folder_loc, file_name):
    OutPutLoc= 'C:/Users/mkahs/Repository/IBR-main/IBR-main/NEW/image_processed_pdf/'
    img_folder_list= input_folder_loc+"/*.jpg"
    imagelist = sorted(glob.glob(img_folder_list), key=os.path.getmtime)
    pdf= FPDF()
    out_pdf_name= OutPutLoc+file_name
    with open(out_pdf_name,"wb") as f:
        f.write(img2pdf.convert(imagelist))
    return out_pdf_name

In [11]:
########### PDF UPLOAD AS INPUT #######################   

# s= 'CAMC_IB_Dana Keen_Claim # 21755930880.pdf'
s= 'C:/Users/mkahs/Repository/IBR-main/IBR-main/NEW/pdf_uploads/CAMC_IB_Dana Keen_Claim # 21755930880.pdf'

########### PDF TO IMAGE PROCESSING #######################    
processed_image_loc, pdf_name = pdf_to_image(s)
print(processed_image_loc)
print(pdf_name)
########### IMAGE TO PDF PROCESSING #######################
processed_pdf = image_to_pdf(processed_image_loc, pdf_name)
print(processed_pdf)
########### PROCESSED PDF TO SEARCHABLE PDF #######################
clean_searchable_pdf_lines, timestr= PDF_searchable_lines(pdf_name )
print(timestr)
# clean_searchable_pdf_lines= PDF_searchable_lines(s)

########### INPUT SEARCH TERM INTO SEARCHABLE PDF #######################
List_check=["0272", "0278","301","0410","0730","0460"]
search_word = "0250"

cost_list_1, total_cost_1, search_df_1 = Search_in_PDF(search_word)
# search_df_1.to_csv('PDF_name_Search_86.csv', index = False)

CAMC_IB_Dana Keen_Claim # 21755930880.pdf
CAMC_IB_Dana Keen_Claim # 21755930880
New File Transfer........
C:/Users/mkahs/Repository/IBR-main/IBR-main/NEW/pdf_image_temp/CAMC_IB_Dana Keen_Claim # 21755930880.pdf/CAMC_IB_Dana Keen_Claim # 21755930880_0.jpg


C:\Users\mkahs\anaconda3\envs\IBR_DEMO\lib\site-packages\ipykernel_launcher.py:480: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\mkahs\anaconda3\envs\IBR_DEMO\lib\site-packages\ipykernel_launcher.py:482: RuntimeWarning: invalid value encountered in double_scalars


126
NOISE REMOVAL DONE
C:/Users/mkahs/Repository/IBR-main/IBR-main/NEW/pdf_image_temp/CAMC_IB_Dana Keen_Claim # 21755930880.pdf/CAMC_IB_Dana Keen_Claim # 21755930880_1.jpg
126
NOISE REMOVAL DONE
C:/Users/mkahs/Repository/IBR-main/IBR-main/NEW/pdf_image_temp/CAMC_IB_Dana Keen_Claim # 21755930880.pdf/CAMC_IB_Dana Keen_Claim # 21755930880_2.jpg
126
NOISE REMOVAL DONE
C:/Users/mkahs/Repository/IBR-main/IBR-main/NEW/pdf_image_temp/CAMC_IB_Dana Keen_Claim # 21755930880.pdf/CAMC_IB_Dana Keen_Claim # 21755930880_3.jpg
126
NOISE REMOVAL DONE
C:/Users/mkahs/Repository/IBR-main/IBR-main/NEW/pdf_image_temp/CAMC_IB_Dana Keen_Claim # 21755930880.pdf/CAMC_IB_Dana Keen_Claim # 21755930880_4.jpg
126
NOISE REMOVAL DONE
C:/Users/mkahs/Repository/IBR-main/IBR-main/NEW/pdf_image_temp/CAMC_IB_Dana Keen_Claim # 21755930880.pdf/CAMC_IB_Dana Keen_Claim # 21755930880_5.jpg
126
NOISE REMOVAL DONE
C:/Users/mkahs/Repository/IBR-main/IBR-main/NEW/pdf_image_temp/CAMC_IB_Dana Keen_Claim # 21755930880.pdf/CAMC_IB_Dana

Scanning contents: 100%|███████████████████████████████████████████████████████████████| 9/9 [00:00<00:00, 10.86page/s]
OCR:  28%|████████████████████▎                                                    | 2.5/9.0 [00:06<00:14,  2.24s/page][tesseract] lots of diacritics - possibly poor OCR
OCR:  83%|████████████████████████████████████████████████████████████▊            | 7.5/9.0 [00:07<00:00,  2.00page/s][tesseract] lots of diacritics - possibly poor OCR
OCR: 100%|█████████████████████████████████████████████████████████████████████████| 9.0/9.0 [00:07<00:00,  1.20page/s]
PDF/A conversion: 100%|████████████████████████████████████████████████████████████████| 9/9 [00:01<00:00,  8.28page/s]
JPEGs: 0image [00:00, ?image/s]
JBIG2: 0item [00:00, ?item/s]


04042022-222602
[16.81, 71.32, 31.74, 18.68, 23.27, 1.0, 1.96, 6.4, 20.01, 4.57, 1.0, 1.0, 12.4, 1.0, 6.2, 18.6, 5.84, 12.4, 12.4, 1.0, 2.92, 7.1, 442.04, 102.48, 102.48, 1.56, 195.59, 18.6, 46.03, 46.03, 1.0, 1.96, 10.01, 18.6, 1.0, 1.96, 6.4, 10.01, 12.1, 6.75, 13.62, 5.84, 41.26, 2.39, 40.92, 528.61, 101.13, 160.12, 15.74, 22.54, 6.95, 76.06, 1.0, 1.96, 12.4, 2.26, 13.56, 15.12, 28.27, 1.0, 16.81, 102.48, 102.48, 5.84, 20.69, 28.27, 16.35, 27.88, 27.48, 20.69, 27.11, 4.67, 4.39, 5.84, 16.81, 0, 11.58, 1.0, 10.06, 1.0, 4.87, 1.23, 4.87, 11.58, 4.87, 1.0, 28.27, 4.87, 16.81, 4.87, 76.06, 11.58, 1.0, 10.27, 1.0, 10.06, 1.0, 15.12, 3.07, 6.17, 16.81, 11.58, 8.8, 102.48, 28.27, 19.21, 11.58, 1.0, 10.27, 10.06, 1.0, 15.12, 1.0, 11.58, 1.0, 11.58, 1.09, 11.58, 1.0, 10.27, 1.0, 10.06, 1.0, 1.39, 11.58, 1.09, 1.39, 1.0, 1.0, 1.0, 1.09, 11.58, 11.58, 1.0, 10.27, 1.0, 1.0, 10.06, 1.0, 0, 1.0, 1.09, 1.09, 1.0, 10.27, 1.0, 1.0, 10.06, 1.0, 1.0, 1.09]
Total Amount :  3456.94


In [5]:
search_df_1.head()

Position Search_Term    Date                  Description  Amount  \
0  Page-2_Line-27  0250        021221  AMIODARONE  500  MG/D5wW     16.81    
1  Page-2_Line-28  0250        021221  EPINEPHRINE  4  MG/D5w       71.32    
2  Page-2_Line-29  0250        021221  INSULIN  REGULAR  100  UNIT  31.74    
3  Page-2_Line-30  0250        021221  NOREPINEPHRINE  (LEVOPHED    18.68    
4  Page-2_Line-31  0250        021221  DNU  DEXMEDETOMIDINE         23.27    

  FLAG  \
0        
1        
2        
3        
4        

                                                                  Line_info  
0  Page-2_Line-27  : 0250  |AMIODARONE  500  MG/D5wW  25  021221  1  16.81   
1  Page-2_Line-28  : 0250  |EPINEPHRINE  4  MG/D5w  250  021221  1  71.32    
2  Page-2_Line-29  : 0250  INSULIN  REGULAR  100  UNIT  021221  1  31.74     
3  Page-2_Line-30  : 0250  NOREPINEPHRINE  (LEVOPHED  021221  1  18.68       
4  Page-2_Line-31  : 0250  |DNU  DEXMEDETOMIDINE  400  021221  1  23.27

In [14]:
save_SearchTerm_DF_to_CSV(pdf_name, search_word, search_df_1, timestr)

Saved as: -- C:/Users/mkahs/Repository/IBR-main/IBR-main/NEW/OUTPUT_CSV_TEMP/RAW_Search_0250_CAMC_IB_Dana Keen_Claim # 21755930880.pdf_04042022-222602.csv
